## Bounds and constraints

In this exercise you learn how to use bounds and simple constraints. 

Note that we will just scratch the surface of the topic. Look at the resources for more information. 

## Resources

- [How to specify bounds](https://estimagic.readthedocs.io/en/stable/how_to_guides/optimization/how_to_specify_bounds.html)
- [How to use constraints](https://estimagic.readthedocs.io/en/stable/how_to_guides/optimization/how_to_specify_constraints.html)
- [Background: How constraints are implemented](https://estimagic.readthedocs.io/en/stable/explanations/optimization/implementation_of_constraints.html)

## Example

We reproduce the example from previous exercises for convenience

In [1]:
import estimagic as em 
import numpy as np

def criterion(x):
    first = (x["a"] - np.pi) ** 4 
    second =  np.linalg.norm(x["b"] - np.arange(3))
    third = np.linalg.norm(x["c"] - np.eye(2))
    return first + second + third
    
    
start_params = {
    "a": 1,
    "b": np.ones(3),
    "c": np.ones((2, 2))
}

In [2]:
res = em.minimize(
    criterion=criterion,
    params=start_params,
    algorithm="nlopt_bobyqa",
)

res.params

{'a': 2.743115791004308,
 'b': array([7.20526405e-06, 1.00000542e+00, 1.99999951e+00]),
 'c': array([[ 1.00000752e+00, -1.83976834e-06],
        [ 5.75654246e-06,  1.00000339e+00]])}

## Task 1: Bounds

Repeat the optimization with an upper bounds of 2.0 on `a` and a lower bound of 0.5 for all entries in `b`

## Solution 1:

In [3]:
res = em.minimize(
    criterion=criterion,
    params=start_params,
    algorithm="nlopt_bobyqa",
    lower_bounds={"b": 0.5 * np.ones_like(start_params["b"])},
    upper_bounds={"a": 2.0},
)

res.params

{'a': 2.0,
 'b': array([0.5       , 1.11950355, 1.82926046]),
 'c': array([[1.00000104e+00, 1.57485251e-06],
        [5.38584210e-06, 1.00000320e+00]])}

## Task 2: Fixing parameters

Remove the bounds but now fix the parameter `a` as well as the top right entry in `c` (i.e. `x["c"][0, 1]`) at their start value

## Solution 2:

In [4]:
res = em.minimize(
    criterion=criterion,
    params=start_params,
    algorithm="nlopt_bobyqa",
    constraints=[
        {"type": "fixed", "selector": lambda x: x["a"]},
        {"type": "fixed", "selector": lambda x: x["c"][0, 1]},
    ],
)

res.params

{'a': 1.0,
 'b': array([1.30231301e-06, 1.00000089e+00, 2.00000549e+00]),
 'c': array([[1.02551721, 1.        ],
        [0.05069564, 0.9944641 ]])}

## Optional: Play around with more constraints

Look at the [documentation](https://estimagic.readthedocs.io/en/stable/how_to_guides/optimization/how_to_specify_constraints.html) and impose the constraint that the parameters in `"c"` sum to 1.

## Solution:

In [5]:
res = em.minimize(
    criterion=criterion,
    params=start_params,
    algorithm="nlopt_bobyqa",
    constraints=[
        {"type":  "linear", "selector": lambda x: x["c"], "value": 1, "weights": 1.0}
    ],
)

res.params

InvalidParamsError: A constraint of type 'linear' is not fulfilled in params. Equality condition of linear constraint violated. The names of the involved parameters are:
['c_0_0', 'c_0_1', 'c_1_0', 'c_1_1']
The relevant constraint is:
{'type': 'linear', 'selector': <function <lambda> at 0x7f2f50ebf0d0>, 'value': 1, 'weights': 4    1.0
5    1.0
6    1.0
7    1.0
dtype: float64, 'index': array([4, 5, 6, 7])}.